# multiscale model fitting for Toy2a

Toy2a is a simplified version of toy2

### start with initalizing many things

In [1]:
# imports
import os
# import sys
import torch
# import pickle
import numpy as np
import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm
# import time
import math

# module_path = os.path.abspath(os.path.join('../src/'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
    
    
# import torch_cae_multilevel_V4 as net
import ResNet2 as tnet
# import utils

In [2]:
# paths
data_dir = './data/toy2a'
model_dir = './models/toy2a'
result_dir = './result/toy2a'

#load data
train_data = torch.tensor(np.load(os.path.join(data_dir, 'train_data.npy')))
val_data = torch.tensor(np.load(os.path.join(data_dir, 'val_data.npy')))
test_data = torch.tensor(np.load(os.path.join(data_dir, 'test_data.npy')))

data_of_sizes = {}
current_size = 2
unresolved_dict = {}
model_keep = list()
model_used_dict = {}


In [3]:
#testing dataset new structure
dt = 1
step_size = 32
n_forward = 5
dataset = tnet.DataSet(torch.flatten(train_data,2,3), torch.flatten(val_data,2,3), torch.flatten(test_data,2,3), dt, step_size, n_forward)

step_size =  32
x_end_idx =  33
y_start_idx =  64
y_end_idx =  194
range(0, 33, 32)
self.train_x shape =  torch.Size([100, 128])
train_ys shape =  torch.Size([100, 5, 64])


C:\Users\16kat\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
C:\Users\16kat\OneDrive - acornonsite.com\Time_Space_multiscale\remake\utils_time2.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.train_x = torch.tensor(train_data[:, x_idx:x_end_idx:step_size, :]).float().to(self.device)
C:\Users\16kat\OneDrive - acornonsite.com\Time_Space_multiscale\remake\utils_time2.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather t

In [4]:
print(dataset.train_ys.shape)

torch.Size([100, 5, 64])


In [5]:
#functions, will move these to a utils file eventually 
#====================================================================================
# def data_of_size(data,size):
#     """
#     Takes averages to shrink size of data
#     Takes data of size (n_points, dim, dim) and shrinks to size (n_points, size, size)
#     takes averages to shrink
#     """
#     return decrease_to_size(torch.tensor(data).unsqueeze(1), size)[:,0,:,:]
#====================================================================================


def isPowerOfTwo(n):
    """
    checks if n is a power of two
    
    input: n, int
    
    output: boolean
    """
    return (np.ceil(np.log2(n)) == np.floor(np.log2(n)));
#====================================================================================
def shrink(data, low_dim):
    '''
    Shrinks data to certain size; either averages or takes endpoints
    
    inputs:
        data: array of size (n_points, n_timesteps, dim, dim) that will shrink
        low_dim: int, size to shrink to, low_dim must be less than or equal to dim
        
    output:
        data: array of size (n_points, n_timesteps, low_dim, low_dim)
    '''
    
    #check inputs
    assert len(data.shape) == 4
    n_points, n_timesteps, dim, _ = data.shape
    assert dim >= low_dim
    assert isPowerOfTwo(low_dim)
    
    if dim == low_dim: #same size, no change
        return data
    
    while(dim > low_dim):
        #shrink by 1 level until same size
        data = apply_local_op(data.float(), 'cpu', ave=average)
        current_size = data.shape[-1]
        
    return data
#====================================================================================
def ave_one_level(data):
    '''
    takes averages to shrink data 1 level
    
    inputs:
        data: tensor of size (n_points, n_timesteps, dim, dim) that will shrink
        
    output:
        processed data: tensor of size (n_points, n_timesteps, dim/2, dim/2)
    '''
    device = 'cpu'
    if not torch.is_tensor(data): #needs to be a tensor
        data = torch.tensor(data)
        
    print("data shape = ", data.shape)
    assert len(data.shape) == 4
#     if data.shape != 4:
#         print("data.shape = ", data.shape)
#         print("data.shape should be of length 4")
    n_points, n_timesteps, dim, _ = data.shape
    
    #dim needs to be even 
    assert dim % 2 == 0
    
    data_right_size = torch.flatten(data, 0,1).unsqueeze(1).float()
    
#     n = min(in_channels, out_channels)
    op = torch.nn.Conv2d(1, 1, 2, stride=2, padding=0).to(device)
   
    op.weight.data = torch.zeros(op.weight.data.size()).to(device)
    op.bias.data = torch.zeros(op.bias.data.size()).to(device)
    op.weight.data[0,0, :, :] = torch.ones(op.weight.data[0,0, :, :].size()).to(device) / 4

    # make them non-trainable
    for param in op.parameters():
        param.requires_grad = False
        
    print("Transforming")
        
    shrunk = op(data_right_size)
    
    print("reshape to print")
    
    return shrunk.squeeze(1).reshape((n_points, n_timesteps, dim//2, dim//2))



In [6]:
print(train_data.shape)
processed = ave_one_level(train_data)
print(processed.shape)

torch.Size([100, 500, 8, 8])
data shape =  torch.Size([100, 500, 8, 8])
Transforming
reshape to print
torch.Size([100, 500, 4, 4])


In [7]:

#make a dictionary with train data of every size 128->1
#====================================================================================

def make_dict_all_sizes(data):
    """
    Makes a dictionary of data at every refinedment size from current->1
    
    inputs:
        data: tensor(or array) of size (n_points, n_timesteps, dim, dim)
        
    outputs: 
        dic: dictionary of tensors. Keys are dim size, tensors are size (n_points, n_timesteps, dim, dim)
    
    """
    
    n_points, n_timesteps, dim, _ = data.shape
    
    if not torch.is_tensor(data): #needs to be a tensor
        data = torch.tensor(data)
        
    assert isPowerOfTwo(dim)
        
    dic = {str(dim): data}
    
    for i in range(int(np.log2(dim))):
        #decrease
        print("i = ", i)
        data = ave_one_level(data)
        dic[str(data.shape[-1])] = data
    
    print(dic.keys())
    
    return dic
#====================================================================================


In [8]:
train_dict = make_dict_all_sizes(train_data)
val_dict = make_dict_all_sizes(val_data)

i =  0
data shape =  torch.Size([100, 500, 8, 8])
Transforming
reshape to print
i =  1
data shape =  torch.Size([100, 500, 4, 4])
Transforming
reshape to print
i =  2
data shape =  torch.Size([100, 500, 2, 2])
Transforming
reshape to print
dict_keys(['8', '4', '2', '1'])
i =  0
data shape =  torch.Size([10, 500, 8, 8])
Transforming
reshape to print
i =  1
data shape =  torch.Size([10, 500, 4, 4])
Transforming
reshape to print
i =  2
data shape =  torch.Size([10, 500, 2, 2])
Transforming
reshape to print
dict_keys(['8', '4', '2', '1'])


In [9]:
#====================================================================================
def train_one_timestep(step_size, train_data, val_data, test_data, current_size, 
                       dt = 1, n_forward = 5, noise=0, make_new = False, dont_train = True, 
                       lr = 1e-3, max_epochs = 10000, batch_size = 50,threshold = 1e-4, 
                       model_dir = './models/toy2',i=None, j = None,print_every=1000):

    """
    fits or loads model at 1 timestep
    
    inputs:
        step_size: int 
        train_data: tensor size (n_points, n_timesteps, dim**2) 
        val_data:tensor size (n_val_points, n_timesteps, dim**2) 
        test_data:tensor size (n_test_points, n_timesteps, dim**2) 
        current_size: int, only used in file naming
        dt = 1: float
        n_forward = 5: int, number of steps to consider during training
        noise=0: float, level of noise, (right now just used in file naming)
        make_new = False: boolean, whether or not to make a new model if old already exists
        dont_train = True: boolean, whether or not to train more if model loaded
        lr = 1e-3: float, learning rate
        max_epochs = 10000: int 
        batch_size = 50: int
        threshold=1e-4: float, stop training when validation gets below threshold
         
    
    outputs:
        model_time: ResNet object of trained model. Also saved
    """
    if (i is not None) and (j is not None):
        
        model_name = 'model_L{}_D{}_noise{}_i{}_j{}.pt'.format(current_size,step_size, noise, i, j)
    else:
        model_name = 'model_L{}_D{}_noise{}.pt'.format(current_size,step_size, noise)
    model_path_this = os.path.join(model_dir, model_name)
    
    n_points, n_timesteps, total_dim = train_data.shape
    arch = [total_dim*2, 128, 128, 128, total_dim] 
    
    try: #if we already have a model saved
        if make_new:
            print("Making a new model. Old one deleted. model {}".format(model_name))
            assert False
        model_time = torch.load(model_path_this)
        print("model loaded: ", model_name)
        print("don't train = ", dont_train)
        if dont_train: #just load model, no training
            return model_time
    except:
        print('create model {} ...'.format(model_name))
        model_time = tnet.ResNet(arch=arch, dt=dt, step_size=step_size)

    print("train_data before dataset", train_data.shape)
    dataset = tnet.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)
    print("train_x after", dataset.train_x.shape)
    
#     #plot the inputed data
#     plt.figure()
#     plt.plot(dataset.val_ys[0, :, 0])#, '.')
# #     plt.plot(np.arange(len(dataset.val_ys[point_num, :, i]))*dataset.step_size, dataset.val_ys[point_num, :, i], '.')
# #     plt.plot(torch.flatten(val_data, 2,3)[0,s_size:,0])
#     plt.title('step size = '+str(step_size))
#     #   plt.xlim([0,100])
#     plt.show()
    
    # training
    model_time.train_net(dataset, max_epoch=max_epochs, batch_size=batch_size, lr=lr,
                    model_path=model_path_this,threshold= threshold,print_every=print_every)
    
    return model_time
#====================================================================================

def find_best_timestep(train_data, val_data, test_data, current_size, start_k = 0, largest_k = 7, 
                       dt = 1, n_forward = 5, noise=0, make_new = False, dont_train = True,
                       lr = 1e-3, max_epochs = 10000, batch_size = 50,threshold = 1e-4, 
                       criterion = torch.nn.MSELoss(reduction='none'), model_dir = "./models/toy2",
                       i=None, j = None,print_every= 1000):
    """
    Trains models with different timestep sizes and finds lowest error
    
    inputs:
     n_forward = 5, noise=0, make_new = False, dont_train = False):
    
        train_data: tensor size (n_points, n_timesteps, dim, dim), or  size (n_points, n_timesteps)
        val_data:tensor size (n_val_points, n_timesteps, dim, dim) , or  size (n_val_points, n_timesteps)
        test_data:tensor size (n_test_points, n_timesteps, dim, dim) , or  size (n_test_points, n_timesteps)
        current_size: int, only used in file naming
        start_k = 0: int, smallest timestep will be 2**start_k
        largest_k = 7:int, largest timestep will be 2**largest_k
        dt = 1: float
        n_forward = 5: int, number of steps to consider during training
        noise=0: float, level of noise, (right now just used in file naming)
        make_new = False: boolean, whether or not to make a new model if old already exists
        dont_train = False: boolean, whether or not to train more if model loaded
        lr = 1e-3: float, learning rate
        max_epochs = 10000: int 
        batch_size = 50: int
        threshold=1e-4: float
        criterion = torch.nn.MSELoss(reduction='none'))
         
         
    outputs:
        models: list of ResNet models
        step_sizes: list of ints for the steps_sizes of models 
        mse_list: list of floats, mse of models 
        idx_lowest: int, index value with lowest mse
         
    """
    
    
    #transform data shapes if needed
    if(len(train_data.shape)== 2):
        train_data = train_data.unsqueeze(2).unsqueeze(3)
        val_data = val_data.unsqueeze(2).unsqueeze(3)
        test_data = test_data.unsqueeze(2).unsqueeze(3)
    assert(len(train_data.shape)== 4)
    assert(len(val_data.shape)== 4)
    assert(len(test_data.shape)== 4)
    
    models = list()
    step_sizes = list()
    n_forward_list = list()
    mse_lowest = 1e10 #big number
    mse_list = list()
    mse_less = 0
    idx_lowest = -1
    
    #make data flat to right dim (n_points, n_timesteps, dim**2)
    train_data = torch.flatten(train_data, 2,3)
    val_data = torch.flatten(val_data, 2,3)
    test_data = torch.flatten(test_data, 2,3)
    
    n_points, n_timesteps, total_dim = train_data.shape
    
    
    
    
    for idx, k in enumerate(range(start_k, largest_k)):
        step_size = 2**k
        step_sizes.append(step_size)
        
        #going to make n_forward the max if can be 
#         n_forward = int((np.floor(n_timesteps/step_size)-1)/2)
#         n_forward_list.append(n_forward)
#         print("n_forward = ", n_forward)
        
        print("train_data shape = ", train_data.shape)
        model_time = train_one_timestep(step_size, train_data, val_data, test_data, current_size, 
                                        make_new = make_new, dont_train = dont_train,i=i, j=j, 
                                        n_forward=n_forward, max_epochs=max_epochs,model_dir=model_dir, print_every = print_every)
        
        
        models.append(model_time)
    
        #find error
        
        y_preds = model_time.uni_scale_forecast(val_data[:, 0, :].float(), n_steps=n_timesteps-1)
        mse_all = criterion(val_data[:, 1:, :].float(), y_preds).mean(-1)

        mean = mse_all.mean(0).detach().numpy()
#         print(mean.shape)
        mse_less = mean.mean()
        mse_list.append(mse_less)

        print("mse_lowest = ", mse_lowest)
        print("mse_less= ", mse_less)
        
        if (mse_less< mse_lowest) or (math.isnan(mse_lowest)) or (math.isnan(mse_less)):
            mse_lowest = mse_less
            idx_lowest = idx

    return models, step_sizes, mse_list, idx_lowest, n_forward_list
#====================================================================================


In [10]:
# def calculate_loss(model, x, ys, w=1.0):
#         """
#         :param x: x batch, array of size batch_size x n_dim
#         :param ys: ys batch, array of size batch_size x n_steps x n_dim
#         :return: overall loss
#         """
#         batch_size, n_steps, n_dim = ys.size()
#         y_preds = torch.zeros(batch_size, n_steps, n_dim*2).float()#.to(self.device)
#         y_prev = x
#         for t in range(n_steps-1):
#             y_next = model(y_prev)
#             y_preds[:, t, :] = y_next
#             print("y_next shap = = ", y_next.shape )
#             y_prev[:,0] = y_prev[:,1]
#             y_prev[:,1] = y_next[:,0]

#         # compute loss
#         criterion = torch.nn.MSELoss(reduction='none')
#         loss = w * criterion(y_preds, ys).mean() + (1-w) * criterion(y_preds, ys).max()

#         return loss

In [11]:
current_size = 1
train_data1 = torch.flatten(train_dict[str(current_size)], 2,3)
val_data1 = torch.flatten(val_dict[str(current_size)], 2,3)
# test_data1 = torch.flatten(val_dict[str(current_size)], 2,3)
model_time = tnet.ResNet(arch=[2,3,3,4,1], dt=dt, step_size=step_size)
print(model_time.n_dim )

# print("train_data before dataset", train_data.shape)
dataset = tnet.DataSet(train_data1, val_data1, val_data1, dt, step_size, n_forward)
print("dataset ndim = ", dataset.n_dim)
print(dataset.train_x.shape)

# calculate_loss(model_time,dataset.train_x, dataset.train_ys)

model_time.train_net(dataset, 1, 100, w=1.0, lr=1e-3, model_path=None, threshold = 1e-8, print_every=1)

arch -=  [2, 3, 3, 4, 1]
2
step_size =  32
x_end_idx =  33
y_start_idx =  64
y_end_idx =  194
range(0, 33, 32)
self.train_x shape =  torch.Size([100, 2])
train_ys shape =  torch.Size([100, 5, 1])
dataset ndim =  2
torch.Size([100, 2])
self.n_dim=  2
dataset.n_dim =  2
x shape in loss funct =  torch.Size([100, 2])
y_prev =  torch.Size([100, 2])
x shape =  torch.Size([100, 2])
x shape =  torch.Size([100, 3])
x shape =  torch.Size([100, 3])
x shape =  torch.Size([100, 4])
x shape =  torch.Size([100, 1])
xinit shape =  torch.Size([100, 2])
ans shape =  torch.Size([100, 1])
x shape =  torch.Size([100, 2])
x shape =  torch.Size([100, 3])
x shape =  torch.Size([100, 3])
x shape =  torch.Size([100, 4])
x shape =  torch.Size([100, 1])
self._modules['increment'](x_init) shape =  torch.Size([100, 1])
y_next shape =  torch.Size([100, 1])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [15]:
x = torch.tensor([[1,2,3,],[4,5,6]])
print(x.shape)
print(x[:,0].shap)

torch.Size([2, 3])


In [ ]:
current_size = 1
models, step_sizes, mse_list, idx_lowest,n_forward_list = find_best_timestep(train_dict[str(current_size)], 
                                                              val_dict[str(current_size)], 
                                                              val_dict[str(current_size)], current_size,model_dir=model_dir, make_new=True, 
                                                             start_k=2, largest_k = 6)

In [ ]:
dataset.train_x.shape

In [ ]:
1500/50

In [ ]:
# #plot a bunch
# # 
# step_size = 4
# dt =1
# n_forward = int(500/step_size - 1)
# model_name = 'model_L{}_D{}_noise{}.pt'.format(current_size,step_size, 0)
# model_path_this = os.path.join(model_dir, model_name)

# n_points, n_timesteps, total_dim = torch.flatten(train_dict[str(current_size)], 2,3).shape
# arch = [total_dim, 128, 128, 128, total_dim] 


# model_time = tnet.ResNet(arch=arch, dt=dt, step_size=step_size)

# dataset = tnet.DataSet(torch.flatten(train_dict[str(current_size)], 2,3), torch.flatten(val_dict[str(current_size)], 2,3), 
#                        torch.flatten(val_dict[str(current_size)], 2,3), dt, step_size, n_forward)

# #plot the inputed data
# plt.figure()
# plt.plot(dataset.val_ys[0, :, 0])#, '.')
# #     plt.plot(np.arange(len(dataset.val_ys[point_num, :, i]))*dataset.step_size, dataset.val_ys[point_num, :, i], '.')
# #     plt.plot(torch.flatten(val_data, 2,3)[0,s_size:,0])
# plt.title('step size = '+str(step_size)+" start")
# #   plt.xlim([0,100])
# plt.show()

# # training
# for i in range(10):
#     model_time.train_net(dataset, batch_size = 32, max_epoch=10,model_path=model_path_this)
#     models = list()
#     models.append(model_time)
#     plt.figure()
# #     plt.plot(dataset.val_ys[0, :, 0])
#     predicted = tnet.multi_scale_forecast(dataset.val_ys[:,0,:],n_forward, models)
#     plt.plot(dataset.val_ys[0,:,0], '.')
#     plt.plot(predicted[0,:,0])
#     plt.title('i = '+str(i))
#     #   plt.xlim([0,100])
#     plt.show()

# # return model_time

In [ ]:
#====================================================================================
def plot_lowest_error(data, model, i = 0):
    """
    Plot data at model, idx
    
    inputs:
        data: tensor of shape (n_points, n_timesteps, dim, dim)
        model: Resnet model to predict on 
        i: int, which validation point to graph
    outputs:
        No returned values, but graph shown
    
    
    """
    data  = torch.flatten(data, 2,3)
    _, total_steps, _ = data.shape
    y_preds = model.uni_scale_forecast(torch.tensor(data[:,0,:]).float(), n_steps=total_steps-1)
    plt.plot(y_preds[i,:,0], label = "Predicted")
    plt.plot(data[i,1:,0], label = "Truth")
    plt.ylim([-.1, 1.1])
    plt.legend()
    plt.show()
#====================================================================================

print(step_sizes[idx_lowest])    
print(step_sizes, mse_list)
plot_lowest_error(val_dict[str(current_size)], models[idx_lowest], i =2)

# print(train_data.shape)
# dataset = tnet.DataSet(torch.flatten(train_data, 2,3), torch.flatten(val_data, 2,3), 
#                        torch.flatten(val_data, 2,3), 1, step_sizes[idx_lowest], 5)
# dataset.plot_val_data()

In [ ]:
def plot_val_data(dataset, point_num = 0, i = 0, other_plot = None):
        
        plt.figure()
        plt.plot(np.arange(len(dataset.val_ys[point_num, :, i]))*dataset.step_size, dataset.val_ys[point_num, :, i])#, '.')
        if other_plot is not None:
            plt.plot(other_plot)
#             plt.xlim([0,100])
        plt.show()

In [ ]:
# print(dataset.train_x.shape)
# print(128**2)
train_data = train_dict['1']
val_data = val_dict['1']
plt.plot(val_data[0,:,0,0])
print(train_data.shape)
s_size = 8
dataset = tnet.DataSet(torch.flatten(train_data, 2,3), torch.flatten(val_data, 2,3), 
                       torch.flatten(val_data, 2,3), 1, s_size, int(np.floor(499/s_size)))
# plot_val_data(dataset, other_plot=torch.flatten(val_data, 2,3)[0,s_size:,0])
point_num = 0
i = 0
plt.figure()
plt.plot(np.arange(len(dataset.val_ys[point_num, :, i]))*dataset.step_size, dataset.val_ys[point_num, :, i])#, '.')
plt.plot(np.arange(len(dataset.val_ys[point_num, :, i]))*dataset.step_size, dataset.val_ys[point_num, :, i], '.')
plt.plot(torch.flatten(val_data, 2,3)[0,s_size:,0])
#   plt.xlim([0,100])
plt.show()

In [ ]:
step_size = 32
n_forward = int(500/step_size - 1)
current_size = 1
train_data = train_dict[str(current_size)]
val_data = val_dict[str(current_size)]
model_time = train_one_timestep(step_size, torch.flatten(train_data, 2,3), 
                                         torch.flatten(val_data, 2,3),  torch.flatten(val_data, 2,3), 
                                         1, n_forward=n_forward,  max_epochs = 1000,)
dataset = tnet.DataSet(torch.flatten(train_data, 2,3), torch.flatten(val_data, 2,3), 
                       torch.flatten(val_data, 2,3), 1, step_size, n_forward)
# print(model_time(dataset.val_ys[:,0,:],n_forward).shape)
models = list()
models.append(model_time)
predicted = tnet.multi_scale_forecast(dataset.val_ys[:,0,:],n_forward, models)
plt.plot(dataset.val_ys[0,:,0], '.')
plt.plot(predicted[0,:,0])
# plt.xlim([0,500])
plt.show()

In [ ]:
#====================================================================================
def find_error_4(data, model, truth_data, tol = 1e-5):
    """
    Find error over the 4 squares 
    
    inputs:
        data: tensor of size (n_points, n_timesteps, dim, dim) to be predicted or size (n_points, n_timesteps)
        model: Resnet object to predict data on
        truth_data: tensor of size (n_points, n_timesteps, dim_larger, dim_larger) compared on 
        tol = 1e-5: tolerance level to mark points as resolved or not
        criterion = torch.nn.MSELoss(reduction='none')
        
    outputs:
        resolved: boolean whether complete area is resolved or not
        loss: array of floats for size (dim, dim) with mse of each square
        unresolved: array of booleans, whether that part is resolved or not. (1 unresolved, 0 resolved)
    """
    if(len(data.shape))==2:
        data = data.unsqueeze(2).unsqueeze(3)
    assert len(data.shape) == 4
    n_points, n_timesteps, dim, _ = data.shape
    data  = torch.flatten(data, 2,3)
    y_preds = model.uni_scale_forecast(torch.tensor(data[:,0,:]).float(), n_steps=n_timesteps-1).reshape(( n_points, n_timesteps-1, dim,dim))
    
    _,_, truth_dim, _ = truth_data.shape
    assert truth_dim >= dim
    
    loss = mse(y_preds, truth_data[:,1:])
    
    resolved =  loss.max() <= tol
    unresolved_array = torch.tensor(loss <= tol)
    
    return resolved, loss, 1-unresolved_array.float()



#====================================================================================    
    
def mse(data1, data2):
    """
    Finds Mean Squared Error between data1 and data2
    
    inputs:
        data1: tensor of shape (n_points, n_timestep, dim1, dim1)
        data2: tensor of shape (n_points, n_timestep, dim2, dim2)
        
    output:
        mse: array of size (min_dim, min_dim) with mse 
    
    """
    #find bigger dim
    size1 = data1.shape[-1]
    size2 = data2.shape[-1]
    size_max = max(size1, size2)
    
    #grow to save sizes and find mse
    mse = np.mean((grow(data1, size_max) - grow(data2, size_max))**2, axis = (0, 1))
    return mse
#====================================================================================
    
def grow(data, dim_full=128):
    '''
    Grow tensor from any size to a bigger size
    inputs: 
        data: tensor to grow, size (n_points, n_timesteps, dim_small, dim_small)
        dim_full = 128: int of size to grow data to

    outputs:
        data_full: tensor size (n_points, n_timesteps, size_full, size_full)
    '''
    n_points, n_timesteps, dim_small, _ = data.shape 
    assert dim_full % dim_small == 0 #need small to be multiple of full

    divide = dim_full // dim_small

    data_full = np.zeros((n_points, n_timesteps, dim_full,dim_full))
    for i in range(dim_small):
        for j in range(dim_small):
            repeated = np.repeat(np.repeat(data[:,:,i,j].reshape(n_points,n_timesteps,1,1), divide, axis = 2), divide, axis = 3)
            data_full[:,:,i*divide:(i+1)*divide, j*divide:(j+1)*divide] = repeated
    return data_full
#====================================================================================


resolved, loss, unresolved_list = find_error_4(val_dict['1'], models[idx_lowest], val_dict['2'])
print(loss.shape)
print(loss)

In [ ]:

unresolved_dict[str(current_size)] = torch.tensor(unresolved_list)

print(unresolved_list)

In [ ]:
next_train_data = unresolved_list * train_dict[str(current_size*2)]
print(next_train_data.shape)
plt.imshow(next_train_data[0,0])
plt.colorbar()
plt.show()

In [ ]:
model_keep.append(models[idx_lowest])
model_used_dict[str(current_size)] = [[0]]

In [ ]:
#====================================================================================
def find_error_1(data, model, tol = 1e-5):
    """
    Find error over the 1 square
    
    inputs:
        data: tensor of size (n_points, n_timesteps, dim, dim) to be predicted
        model: Resnet object to predict data on
        tol = 1e-5: tolerance level to mark points as resolved or not
        criterion = torch.nn.MSELoss(reduction='none')
        
    outputs:
        loss: float of mse
        resolved: boolean whether resolved or not
    """
    n_points, n_timesteps  = data.shape
    dim = 1
    data_input  = data.unsqueeze(2)
    y_preds = model.uni_scale_forecast(torch.tensor(data_input[:,0,:]).float(), n_steps=n_timesteps-1).reshape(( n_points, n_timesteps-1, dim,dim))
    data1 = data[:,1:]
    data2 = y_preds[:,:,0,0]
#     print()
    loss = torch.mean((data1-data2)**2)#mse(y_preds, data[:,1:])
    
#     print(loss)
    
    return loss, loss <= tol

#====================================================================================


current_size = 2
next_train_data = unresolved_list * train_dict[str(current_size)]

model_idx_list = np.ones((current_size, current_size))*(-1) #start with all -1

for i in range(current_size):
    for j in range(current_size):
        data_this = next_train_data[:,:,i,j]
        if (torch.min(data_this) == 0) and (torch.max(data_this) == 0):
            #don't need to do anything is model is resolved
            continue
        else:
        #see if the error is low enough on already made model
            for m, model in enumerate(model_keep):
                loss, resolved = find_error_1(data_this, model)
                step_size = model.step_size
                print("loss = ", loss)
                print("step_size = ", step_size)
                if resolved:
                    model_idx_list[i,j] == m
                    break
                else:
                    pass
            if not resolved:
                i = 0
                j = 1
                k = int(np.log2(step_size))
                print("k = ", k)
                print("train_dict[str(current_size)][:,:,i,j] shape = ", train_dict[str(current_size)][:,:,i,j].shape)
                #if no model good, train new model
                models, step_sizes, mse_list, idx_lowest = find_best_timestep(train_dict[str(current_size)][:,:,i,j], 
                                                              val_dict[str(current_size)][:,:,i,j], 
                                                              val_dict[str(current_size)][:,:,i,j], current_size,model_dir=model_dir,
                                                              i=i, j=j, start_k = max(0,k-1), largest_k = k+2)
                
                vbnm
                resolved, loss, unresolved_list = find_error_4(val_dict[str(current_size)][:,:,i,j], 
                                                               models[idx_lowest], 
                                                               val_dict[str(current_size)][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size])
                model_keep.append(models[idx_lowest])
                model_idx_list[i,j] == len(model_keep) #last model will be the one for this square
            
#             predicted = model.uni_scale_forecast(torch.tensor(data[:,0,:]).float(), n_steps=n_timesteps-1).reshape((  n_points, n_timesteps-1, dim,dim))

In [ ]:
print(step_sizes, mse_list, idx_lowest)
resolved, loss, unresolved_list = find_error_4(val_dict[str(current_size)][:,:,i,j], 
                                                               models[idx_lowest], 
                                                               val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size])
print(loss)    

In [ ]:
model =  models[idx_lowest]
print(idx_lowest)
n_timesteps = 500
n_points = 10
dim = 1
# plt.plot(model(val_dict[str(current_size)][:,:,i,j].unsqueeze(2).unsqueeze(3))[0,:,0,0].detach().numpy(), label = "predicted")
print(val_dict[str(current_size)][:,0,i,j].unsqueeze(1).shape)
val_data_this = val_dict[str(current_size)][:,0,i,j].unsqueeze(1)
predicted = model.uni_scale_forecast(val_data_this, n_steps=n_timesteps-1)
print(predicted.shape)
predicted_reshape = predicted.reshape((  n_points, n_timesteps-1, dim,dim))
plt.plot(predicted_reshape[0,:,0], label = "predicted")
print(val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size].shape)
# plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,0], label = "Truth")

plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,1], label = "Truth")
plt.legend()

In [ ]:
plt.imshow(val_dict[str(current_size*2)][0,0])
plt.colorbar()

In [ ]:
step_size = (16+32)/2
print(step_size)
model = train_one_timestep(int(28), train_dict[str(current_size)][:,:,i,j].unsqueeze(2), 
                           val_dict[str(current_size)][:,:,i,j].unsqueeze(2), 
                           val_dict[str(current_size)][:,:,i,j].unsqueeze(2), current_size)
#                        dt = 1, n_forward = 5, noise=0, make_new = False, dont_train = True, 
#                        lr = 1e-3, max_epochs = 10000, batch_size = 50,threshold = 1e-4, 
#                        model_dir = './models/toy2',i=None, j = None):
    
#     train_dict[str(current_size)][:,:,i,j], 
#                                                               val_dict[str(current_size)][:,:,i,j], 
#                                                               val_dict[str(current_size)][:,:,i,j], current_size,model_dir=model_dir, 
#                                                               i=i, j=j, start_k = max(0,k-1), largest_k = k+2)

In [ ]:
i = 0
j = 1
resolved, loss, unresolved_list = find_error_4(val_dict[str(current_size)][:,:,i,j], 
                                                               model, 
                                                               val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size])
print(loss)

In [ ]:
# model =  models[idx_lowest]
print(idx_lowest)
n_timesteps = 500
n_points = 10
dim = 1
# plt.plot(model(val_dict[str(current_size)][:,:,i,j].unsqueeze(2).unsqueeze(3))[0,:,0,0].detach().numpy(), label = "predicted")
print(val_dict[str(current_size)][:,0,i,j].unsqueeze(1).shape)
val_data_this = val_dict[str(current_size)][:,0,i,j].unsqueeze(1)
predicted = model.uni_scale_forecast(val_data_this, n_steps=n_timesteps-1)
print(predicted.shape)
predicted_reshape = predicted.reshape((  n_points, n_timesteps-1, dim,dim))
plt.plot(predicted_reshape[0,:,0], label = "predicted")
print(val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size].shape)
# plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,0], label = "Truth")

plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,1], label = "Truth")
plt.legend()